In [1]:
from pydantic import BaseModel, Field
from typing import Dict, Any
from typing import Dict, Any

class EventPayload(BaseModel):
    """Information about a requested action to be broadcast to the system."""
    intent: str = Field(description="The action, e.g., CREATE, UPDATE, DELETE")
    target: str = Field(description="The entity type, e.g., JOB_ENTITY, TASK_ENTITY")
    params: Dict[str, Any] = Field(description="Key-value pairs of extracted data like client or date")
    raw_input: str = Field(description="The original user message")

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# Bind the schema to the LLM
structured_llm = llm.with_structured_output(EventPayload)

# Simple function to generate the event
def get_event_from_input(user_query: str):
    return structured_llm.invoke(user_query)




In [10]:
result = get_event_from_input("Hey, I want to create a job for Coke")
print(result.model_dump_json())

{"intent":"CREATE","target":"JOB_ENTITY","params":{"client":"Coke"},"raw_input":"create a job for Coke"}


In [12]:
result = get_event_from_input("Hey, I want to create a job for Coke with due date 20Dec 2026 and assign it to Ritik.kumar@abc.com")
print(result.model_dump_json())

{"intent":"CREATE","target":"JOB_ENTITY","params":{"assignee":"Ritik.kumar@abc.com","client":"Coke","due_date":"20Dec2026"},"raw_input":"Create a job for Coke with due date 20Dec 2026 and assign it to Ritik.kumar@abc.com"}
